In [1]:
# Before beginning this notebook you will need to add the following code to the query tool in pgAdmin: 
# ALTER TABLE counties ADD PRIMARY KEY (fips_date); ALTER TABLE hospitals ADD PRIMARY KEY (fips_date);
# Additionally, you will need to enter your password into the connection_string variable in the config.py file. 
# The place holder text is YOURPASSWORDHERE

In [ ]:
# Import dependencies
import sqlalchemy
from sqlalchemy import create_engine
from config import connection_string
import pandas as pd
import psycopg2 as pg

In [2]:
 # Create engine using the database file
engine = sqlalchemy.create_engine(connection_string)

In [3]:
# Reflect an existing databade into a new model
Base = automap_base()

In [4]:
# Reflect the tables
Base.prepare(engine, reflect=True)

In [5]:
dbConnection = engine.connect();

In [ ]:
merged_df = pd.read_sql("SELECT * FROM counties INNER JOIN hospitals ON counties.fips_date = hospitals.fips_date", dbConnection);